# Train data generator for reverse ranker


In [1]:
import sys
sys.path.insert(0, '/home/jianx/search-exposure/')
import importlib
import torch
from annoy import AnnoyIndex

import forward_ranker.network as network
importlib.reload(network)
import forward_ranker.load_data as load_data
importlib.reload(load_data)
obj_reader = load_data.obj_reader
import forward_ranker.train as train
importlib.reload(train)
generate_sparse = train.generate_sparse

In [2]:
MODEL_PATH = "/home/jianx/data/results/100_1000_1000_0.001_256_10.model"
DEVICE = torch.device("cuda:0")
EMBED_SIZE = 256
TREE_SIZE = 10000

In [3]:
NET = network.DSSM(embed_size=EMBED_SIZE)
NET.load_state_dict(torch.load(MODEL_PATH))
NET.to(DEVICE)
NET.eval()

DSSM(
  (model): Sequential(
    (0): Linear(in_features=100000, out_features=64, bias=True)
    (1): ReLU()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ReLU()
    (10): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=64, out_features=256, bias=True)
  )
)

In [4]:
QUERY_TRAIN_DICT_PATH = "/home/jianx/data/queries_train.dict"
PASSAGE_DICT_PATH = "/home/jianx/data/passages.dict"

QUERY_DICT = obj_reader(QUERY_TRAIN_DICT_PATH)
PASSAGE_DICT = obj_reader(PASSAGE_DICT_PATH)

In [5]:
def generate_annoy_index(net, device, embed_size, query_dict, passage_dict):
    qid_mapping = {}
    pid_mapping = {}
    i = 0
    index = AnnoyIndex(embed_size, 'euclidean')
    for key, value in query_dict.items():
        if len(value) != 0:
            index.add_item(i, net(generate_sparse(value).to(device)).detach().tolist())
            qid_mapping[key] = i
            i += 1
    for key, value in passage_dict.items():
        if len(value) != 0:
            index.add_item(i, net(generate_sparse(value).to(device)).detach().tolist())
            pid_mapping[key] = i
            i+=1
    return index,qid_mapping,pid_mapping

In [12]:
PASSAGE_INDEX = AnnoyIndex(EMBED_SIZE, 'euclidean')
PASSAGE_INDEX.load("/home/jianx/data/annoy/100_passage_index.ann")
PID_MAPPING = obj_reader("/home/jianx/data/annoy/100_pid_map.dict")

In [13]:
print(PASSAGE_INDEX.get_nns_by_vector(NET(generate_sparse([361,7867]).to(DEVICE)).detach().tolist(),1000))



[8431452, 3355020, 7728594, 4469580, 434114, 7093392, 7217580, 233284, 4423510, 6112490, 2921299, 7086849, 7893779, 292816, 6322716, 7882469, 5327771, 1438344, 7200084, 7515089, 4559035, 1836687, 1725278, 4560785, 2510630, 4869331, 8160071, 6262698, 5015856, 7486469, 3709932, 2616512, 8194241, 6400738, 6585937, 4797380, 1883236, 2937548, 8160118, 8040418, 3806352, 4685151, 8418478, 2536548, 4922538, 4072902, 4652768, 4810814, 7836589, 6609098, 876952, 950063, 3722570, 7627922, 1472643, 4365682, 3588802, 3899112, 7593114, 3873383, 1226746, 1005142, 35544, 7038773, 5457271, 2529797, 2567412, 1236935, 8046328, 1207432, 6908919, 2186471, 4208548, 3976415, 1489824, 2851602, 3722566, 1455671, 7978655, 5054734, 3281218, 2072511, 2319209, 1677838, 1702908, 2151280, 3909284, 8816549, 3983970, 412007, 6937199, 414171, 7993608, 5983529, 7793266, 6933173, 1501336, 7094948, 7441758, 1717975, 3871071, 2438475, 1070819, 6818537, 3912621, 5264214, 6705082, 1723592, 2538760, 8174973, 8566103, 6232103, 

In [17]:
print(PID_MAPPING[7728594])


7728453
